In [2]:
import json
from pathlib import Path

import polars as pl
from upwork_scrape.scrape_data import JobsScraper
from upwork_scrape.write_to_db import write_to_db

In [ ]:
frame_works=["react","svelte","angular","vue","astro","nextjs","nestjs"]
langs=["python","php","rust","javascript"]
low_code=["wordpress","webflow"]
topics=["etl","data processing", "data science", "web development", "programming","software engineer"]
all_queries=langs+low_code+frame_works+topics

In [ ]:
JobsScraper.scrape(all_queries[15:])

In [3]:
final_df=(
  pl.concat([pl.read_csv(query,schema={"uid": pl.Int64,
        "url": pl.String,
        "title": pl.String,
        "description": pl.String,
        "created_at": pl.Float64,
        "skills": pl.String,
        "pay_type": pl.String,
        "experience_level": pl.String,
        "time_estimate": pl.String,
        "budget": pl.Int64,
        "max_rate": pl.Int64,
        "min_rate": pl.Int64,
        "search_query": pl.String}) for query in Path.cwd().rglob("data/*.csv")])
    .unique(("uid","url"))
    .with_columns(skills=pl.col("skills").map_elements(lambda text: json.loads(text.replace("'", '"')),pl.List(pl.Utf8)),
                  pay_type=pl.col("pay_type").str.to_lowercase(),
                  created_at=pl.from_epoch("created_at").dt.convert_time_zone("America/New_York")
                  )
    # .with_columns(day_name=pl.col("created_at").dt.weekday().replace_strict(NUM_TO_NAME_DOW))
)

In [4]:
write_to_db(final_df)